In [4]:
import os
import seaborn as sns
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [6]:
def softmax(x, T, b=0):
    x = x / T + b
    f = np.exp(x - np.max(x, axis = 0))  # shift values
    return f / f.sum(axis = 0)

In [7]:
cifar_resultsdir = '/vol/biomedic3/zl9518/ModelEvaluation/LDAM-DRW/cifar10results/'
cnn_pred = pd.read_csv(cifar_resultsdir + 'predictions_val.csv')

In [8]:
kcls = 10
targets_all = np.array(cnn_pred[['target_0', 'target_1', 'target_2', 'target_3', 'target_4', 'target_5', 'target_6', 'target_7', 'target_8', 'target_9']])
logit_all = np.array(cnn_pred[['logit_0', 'logit_1', 'logit_2', 'logit_3', 'logit_4', 'logit_5', 'logit_6', 'logit_7', 'logit_8', 'logit_9']])
preds_all_argmax = np.argmax(logit_all, axis = 1)
# acc
target_class = np.argmax(targets_all, axis = 1)
pred_class = np.argmax(logit_all, axis = 1)
acc = np.sum(pred_class == target_class) / len(target_class)
prob = softmax(logit_all.transpose(), T = 1).transpose()
probmax = np.max(prob, axis = 1)
prob_Topt = softmax(logit_all.transpose(), T = 3.0).transpose()
prob_Toptmax = np.max(prob_Topt, axis = 1)
print(acc)
print(np.mean(probmax))
print(np.mean(prob_Toptmax))

0.7054
0.9106864551906859
0.7072816176222978


In [9]:
preacts = logit_all
labels = target_class

In [2]:
# prepare:
# -> preacts. N x C
# -> labels. N
def eval_func(x):
    ws = np.array(x[:int(len(x)/2)])
    bs = np.array(x[int(len(x)/2):]) 

    vs_logits = preacts/ws[None,:] + bs[None,:]
    exp_vs_logits = np.exp(vs_logits)
    sum_exp = np.sum(exp_vs_logits, axis=1, keepdims=True)
    AC = np.mean(np.max(exp_vs_logits/sum_exp, axis=1))
    preds = np.argmax(preacts, axis = 1)
    acc = np.sum(labels == preds) / len(labels)
    MC = np.abs(AC-acc)

    return MC

In [180]:
import scipy
optimization_result = scipy.optimize.minimize(
                      fun=eval_func,
#                       fun=lambda x: eval_func(x)[0],
                      x0=np.array([1.0 for x in range(preacts.shape[1])]
                                  +[0.0 for x in range(preacts.shape[1])]),
                      bounds=[(0,None) for x in range(preacts.shape[1])]
                              +[(None,None) for x in range(preacts.shape[1])],
#                       jac=True,
                      method='L-BFGS-B',
                      tol=1e-07)

In [181]:
optimization_result

     fun: 5.192912766460722e-11
     jac: array([-0.00993638, -0.00979049, -0.01376561,  0.0016993 , -0.00716579,
       -0.04151871, -0.00791228, -0.00620294, -0.01278704, -0.00920676,
        0.00711787, -0.00170159,  0.00068763, -0.01562243, -0.00118715,
        0.01287408,  0.00064462, -0.00283404,  0.0008278 , -0.00080675])
 message: 'Optimization terminated successfully'
    nfev: 204
     nit: 9
    njev: 9
  status: 0
 success: True
       x: array([ 4.61069834e+00,  2.95019820e+00,  2.79173523e+00,  5.90396901e+00,
        3.94892438e+00,  1.66745684e+00,  3.70487555e+00,  3.23876667e+00,
        1.95357580e+00,  2.41730853e+00, -1.49443554e-01,  1.59860198e-02,
        1.87513256e-01, -1.93028018e-01, -5.53625175e-02,  3.24705318e-01,
       -1.38296583e-01, -7.68951271e-02,  8.85976762e-02, -3.78795058e-03])

In [10]:
eval_func([ 4.61069834e+00,  2.95019820e+00,  2.79173523e+00,  5.90396901e+00,
        3.94892438e+00,  1.66745684e+00,  3.70487555e+00,  3.23876667e+00,
        1.95357580e+00,  2.41730853e+00, -1.49443554e-01,  1.59860198e-02,
        1.87513256e-01, -1.93028018e-01, -5.53625175e-02,  3.24705318e-01,
       -1.38296583e-01, -7.68951271e-02,  8.85976762e-02, -3.78795058e-03])

1.9574786236375985e-10

In [17]:
Ts = 3
eval_func([ Ts,  Ts,  Ts,  Ts,
        Ts,  Ts,  Ts,  Ts,
        Ts,  Ts, 0,  0,
        0, 0, 0,  0,
       0, 0,  0, 0])

0.001881617622297771

In [163]:
classratio = [5000, 2997, 1796, 1077, 645, 387, 232, 139, 83, 50]

In [76]:
# prepare:
# -> preacts. N x C
# -> labels. N
def eval_func(x):
    ws = np.array(x[:int(len(x)/2)])
    bs = np.array(x[int(len(x)/2):]) 
    
    ws = np.concatenate((classratio[0]/classratio[0] * ws[None, 0] + np.ones(1),
                         classratio[1]/classratio[0] * ws[None, 0] + np.ones(1),
                         classratio[2]/classratio[0] * ws[None, 0] + np.ones(1),
                         classratio[3]/classratio[0] * ws[None, 0] + np.ones(1),
                         classratio[4]/classratio[0] * ws[None, 0] + np.ones(1),
                         classratio[5]/classratio[0] * ws[None, 0] + np.ones(1),
                         classratio[6]/classratio[0] * ws[None, 0] + np.ones(1),
                         classratio[7]/classratio[0] * ws[None, 0] + np.ones(1),
                         classratio[8]/classratio[0] * ws[None, 0] + np.ones(1),
                         classratio[9]/classratio[0] * ws[None, 0] + np.ones(1)))
    vs_logits = preacts/ws[None,:]
    exp_vs_logits = np.exp(vs_logits)
    sum_exp = np.sum(exp_vs_logits, axis=1, keepdims=True)
    AC = np.mean(np.max(exp_vs_logits/sum_exp, axis=1))
    preds = np.argmax(preacts, axis = 1)
    acc = np.sum(labels == preds) / len(labels)
    MC = np.abs(AC-acc)

    return MC

In [77]:
import scipy
optimization_result = scipy.optimize.minimize(
                      fun=eval_func,
#                       fun=lambda x: eval_func(x)[0],
                      x0=np.array([1.0 for x in range(preacts.shape[1])]
                                  +[0.0 for x in range(preacts.shape[1])]),
                      bounds=[(0,None) for x in range(preacts.shape[1])]
                              +[(None,None) for x in range(preacts.shape[1])],
#                       jac=True,
                      method='L-BFGS-B',
                      tol=1e-07)

In [78]:
optimization_result

      fun: 6.853961842523404e-12
 hess_inv: <20x20 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.00999609, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 693
      nit: 6
     njev: 33
   status: 0
  success: True
        x: array([11.53481921,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [6]:
cifar_resultsdir = '/vol/biomedic3/zl9518/ModelEvaluation/LDAM-DRW/cifar10results/'
cnn_pred = pd.read_csv(cifar_resultsdir + 'predictions_train.csv')

In [14]:
kcls = 10
targets_all = np.array(cnn_pred[['target_0', 'target_1', 'target_2', 'target_3', 'target_4', 'target_5', 'target_6', 'target_7', 'target_8', 'target_9']])
logit_all = np.array(cnn_pred[['logit_0', 'logit_1', 'logit_2', 'logit_3', 'logit_4', 'logit_5', 'logit_6', 'logit_7', 'logit_8', 'logit_9']])
preds_all_argmax = np.argmax(logit_all, axis = 1)
# acc
target_class = np.argmax(targets_all, axis = 1)
pred_class = np.argmax(logit_all, axis = 1)
acc = np.sum(pred_class == target_class) / len(target_class)
prob = softmax(logit_all.transpose(), T = 1).transpose()
probmax = np.max(prob, axis = 1)
prob_Topt = softmax(logit_all.transpose(), T = 0.6).transpose()
prob_Toptmax = np.max(prob_Topt, axis = 1)
print(acc)
print(np.mean(probmax))
print(np.mean(prob_Toptmax))

9.965339351926488e-01
0.9909580019290042
0.9962229754794684


In [25]:
# choose TS for different classes, manually
label = 0
targets_y1 = np.where(preds_all_argmax==label)[0]
pred_class = np.argmax(logit_all, axis = 1)[targets_y1]
target_class = np.argmax(targets_all, axis = 1)[targets_y1]
acc = np.sum(pred_class == target_class) / len(target_class)
prob = softmax(logit_all.transpose(), T = 1).transpose()[targets_y1]
probmax = np.max(prob, axis = 1)
prob_Topt = softmax(logit_all.transpose(), T = 5.0).transpose()[targets_y1]
prob_Toptmax = np.max(prob_Topt, axis = 1)
print('cls ' + str(label))
print(acc)
print(np.mean(probmax))
print(np.mean(prob_Toptmax))
label = 1
targets_y1 = np.where(preds_all_argmax==label)[0]
pred_class = np.argmax(logit_all, axis = 1)[targets_y1]
target_class = np.argmax(targets_all, axis = 1)[targets_y1]
acc = np.sum(pred_class == target_class) / len(target_class)
prob = softmax(logit_all.transpose(), T = 1).transpose()[targets_y1]
probmax = np.max(prob, axis = 1)
prob_Topt = softmax(logit_all.transpose(), T = 5.1).transpose()[targets_y1]
prob_Toptmax = np.max(prob_Topt, axis = 1)
print('cls ' + str(label))
print(acc)
print(np.mean(probmax))
print(np.mean(prob_Toptmax))
label = 2
targets_y1 = np.where(preds_all_argmax==label)[0]
pred_class = np.argmax(logit_all, axis = 1)[targets_y1]
target_class = np.argmax(targets_all, axis = 1)[targets_y1]
acc = np.sum(pred_class == target_class) / len(target_class)
prob = softmax(logit_all.transpose(), T = 1).transpose()[targets_y1]
probmax = np.max(prob, axis = 1)
prob_Topt = softmax(logit_all.transpose(), T = 3.1).transpose()[targets_y1]
prob_Toptmax = np.max(prob_Topt, axis = 1)
print('cls ' + str(label))
print(acc)
print(np.mean(probmax))
print(np.mean(prob_Toptmax))
label = 3
targets_y1 = np.where(preds_all_argmax==label)[0]
pred_class = np.argmax(logit_all, axis = 1)[targets_y1]
target_class = np.argmax(targets_all, axis = 1)[targets_y1]
acc = np.sum(pred_class == target_class) / len(target_class)
prob = softmax(logit_all.transpose(), T = 1).transpose()[targets_y1]
probmax = np.max(prob, axis = 1)
prob_Topt = softmax(logit_all.transpose(), T = 3.8).transpose()[targets_y1]
prob_Toptmax = np.max(prob_Topt, axis = 1)
print('cls ' + str(label))
print(acc)
print(np.mean(probmax))
print(np.mean(prob_Toptmax))
label = 4
targets_y1 = np.where(preds_all_argmax==label)[0]
pred_class = np.argmax(logit_all, axis = 1)[targets_y1]
target_class = np.argmax(targets_all, axis = 1)[targets_y1]
acc = np.sum(pred_class == target_class) / len(target_class)
prob = softmax(logit_all.transpose(), T = 1).transpose()[targets_y1]
probmax = np.max(prob, axis = 1)
prob_Topt = softmax(logit_all.transpose(), T = 2.2).transpose()[targets_y1]
prob_Toptmax = np.max(prob_Topt, axis = 1)
print('cls ' + str(label))
print(acc)
print(np.mean(probmax))
print(np.mean(prob_Toptmax))
label = 5
targets_y1 = np.where(preds_all_argmax==label)[0]
pred_class = np.argmax(logit_all, axis = 1)[targets_y1]
target_class = np.argmax(targets_all, axis = 1)[targets_y1]
acc = np.sum(pred_class == target_class) / len(target_class)
prob = softmax(logit_all.transpose(), T = 1).transpose()[targets_y1]
probmax = np.max(prob, axis = 1)
prob_Topt = softmax(logit_all.transpose(), T = 1.9).transpose()[targets_y1]
prob_Toptmax = np.max(prob_Topt, axis = 1)
print('cls ' + str(label))
print(acc)
print(np.mean(probmax))
print(np.mean(prob_Toptmax))
label = 6
targets_y1 = np.where(preds_all_argmax==label)[0]
pred_class = np.argmax(logit_all, axis = 1)[targets_y1]
target_class = np.argmax(targets_all, axis = 1)[targets_y1]
acc = np.sum(pred_class == target_class) / len(target_class)
prob = softmax(logit_all.transpose(), T = 1).transpose()[targets_y1]
probmax = np.max(prob, axis = 1)
prob_Topt = softmax(logit_all.transpose(), T = 1.1).transpose()[targets_y1]
prob_Toptmax = np.max(prob_Topt, axis = 1)
print('cls ' + str(label))
print(acc)
print(np.mean(probmax))
print(np.mean(prob_Toptmax))
label = 7
targets_y1 = np.where(preds_all_argmax==label)[0]
pred_class = np.argmax(logit_all, axis = 1)[targets_y1]
target_class = np.argmax(targets_all, axis = 1)[targets_y1]
acc = np.sum(pred_class == target_class) / len(target_class)
prob = softmax(logit_all.transpose(), T = 1).transpose()[targets_y1]
probmax = np.max(prob, axis = 1)
prob_Topt = softmax(logit_all.transpose(), T = 0.6).transpose()[targets_y1]
prob_Toptmax = np.max(prob_Topt, axis = 1)
print('cls ' + str(label))
print(acc)
print(np.mean(probmax))
print(np.mean(prob_Toptmax))
label = 8
targets_y1 = np.where(preds_all_argmax==label)[0]
pred_class = np.argmax(logit_all, axis = 1)[targets_y1]
target_class = np.argmax(targets_all, axis = 1)[targets_y1]
acc = np.sum(pred_class == target_class) / len(target_class)
prob = softmax(logit_all.transpose(), T = 1).transpose()[targets_y1]
probmax = np.max(prob, axis = 1)
prob_Topt = softmax(logit_all.transpose(), T = 0.4).transpose()[targets_y1]
prob_Toptmax = np.max(prob_Topt, axis = 1)
print('cls ' + str(label))
print(acc)
print(np.mean(probmax))
print(np.mean(prob_Toptmax))
label = 9
targets_y1 = np.where(preds_all_argmax==label)[0]
pred_class = np.argmax(logit_all, axis = 1)[targets_y1]
target_class = np.argmax(targets_all, axis = 1)[targets_y1]
acc = np.sum(pred_class == target_class) / len(target_class)
prob = softmax(logit_all.transpose(), T = 1).transpose()[targets_y1]
probmax = np.max(prob, axis = 1)
prob_Topt = softmax(logit_all.transpose(), T = 0.2).transpose()[targets_y1]
prob_Toptmax = np.max(prob_Topt, axis = 1)
print('cls ' + str(label))
print(acc)
print(np.mean(probmax))
print(np.mean(prob_Toptmax))

cls 0
0.5130111524163569
0.9225970527876363
0.5174702558349566
cls 1
0.667574931880109
0.9465175121023574
0.6638625247488893
cls 2
0.6723577235772358
0.8975987002607149
0.6747693768252498
cls 3
0.5485674353598882
0.8778312932453294
0.5515075198723262
cls 4
0.7966976264189887
0.9152566261328222
0.7927198522007158
cls 5
0.7688504326328801
0.8755787630858152
0.7727616252283973
cls 6
0.920881971465629
0.9290975108112881
0.9200385890090808
cls 7
0.9548494983277592
0.9130334698715659
0.9526457191968399
cls 8
0.9671717171717171
0.9070707221184109
0.9661278774550892
cls 9
0.9842696629213483
0.9059517630576093
0.9824487077408494


In [151]:
learnT = 3.0
# learnTs = [5.0, 5.1, 3.1, 3.8, 2.2, 1.9, 1.1, 0.6, 0.4, 0.2]
learnThp = 11.53481921
learnTs = [1 + classratio[0]/classratio[0] * learnThp,
           1 + classratio[1]/classratio[0] * learnThp,
           1 + classratio[2]/classratio[0] * learnThp,
           1 + classratio[3]/classratio[0] * learnThp,
           1 + classratio[4]/classratio[0] * learnThp,
           1 + classratio[5]/classratio[0] * learnThp,
           1 + classratio[6]/classratio[0] * learnThp,
           1 + classratio[7]/classratio[0] * learnThp,
           1 + classratio[8]/classratio[0] * learnThp,
           1 + classratio[9]/classratio[0] * learnThp]

In [152]:
learnTs

[12.53481921,
 7.913970634474,
 5.143307060232,
 3.484600057834,
 2.48799167809,
 1.892795006854,
 1.5352156113439999,
 1.3206679740379998,
 1.191477998886,
 1.1153481921]

In [153]:
# bring a test condition here.
acc_results = []
AC_results = []
TS_results = []
CSTS_results = []
cifar_resultsdir = '/vol/biomedic3/zl9518/ModelEvaluation/LDAM-DRW/cifar10results/'
corruptions = ['gaussian_noise', 'shot_noise' , 'impulse_noise', 'defocus_blur', 'glass_blur', 
               'motion_blur', 'zoom_blur', 'snow', 'frost', 'fog', 'brightness', 'contrast' , 'elastic_transform', 
               'pixelate', 'jpeg_compression', 'speckle_noise', 'gaussian_blur', 'spatter', 'saturate']

for cname in tqdm(corruptions):
    csvfilename = cifar_resultsdir + 'predictions_val_' + cname + '.csv'
    cnn_pred_all = pd.read_csv(csvfilename)
    for severity in range(5):
        cnn_pred = cnn_pred_all.iloc[severity * 10000:(severity + 1) * 10000, :]
        
        kcls = 10
        targets_all = np.array(cnn_pred[['target_0', 'target_1', 'target_2', 'target_3', 'target_4', 'target_5', 'target_6', 'target_7', 'target_8', 'target_9']])
        logit_all = np.array(cnn_pred[['logit_0', 'logit_1', 'logit_2', 'logit_3', 'logit_4', 'logit_5', 'logit_6', 'logit_7', 'logit_8', 'logit_9']])
        preds_all_argmax = np.argmax(logit_all, axis = 1)
        # acc
        target_class = np.argmax(targets_all, axis = 1)
        pred_class = np.argmax(logit_all, axis = 1)
        acc = np.sum(pred_class == target_class) / len(target_class)
        prob = softmax(logit_all.transpose(), T = 1).transpose()
        probmax = np.max(prob, axis = 1)
        prob_Topt = softmax(logit_all.transpose(), T = learnT).transpose()
        prob_Toptmax = np.max(prob_Topt, axis = 1)
        acc_results.append(acc)
        AC_results.append(np.mean(probmax))
        TS_results.append(np.mean(prob_Toptmax))
        targets_all = []
        preds_class_all = []
        for label in range(kcls):
            preds_all = softmax(logit_all.transpose(), T = learnTs[label]).transpose()
            preds_all_max = np.max(preds_all, axis = 1)
            targets_y1 = np.where(preds_all_argmax==label)[0]
            preds_class = preds_all_max[targets_y1]

            preds_class_all = np.concatenate((preds_class_all, preds_class), axis=0)
        CSTS_results.append(np.mean(preds_class_all))

100%|████████████████████████████| 19/19 [00:04<00:00,  4.37it/s]


In [154]:
print('AC_results:')
print(np.mean(np.abs(np.array(acc_results)-np.array(AC_results))))
print('TS_results:')
print(np.mean(np.abs(np.array(acc_results)-np.array(TS_results))))
print('CSTS_results:')
print(np.mean(np.abs(np.array(acc_results)-np.array(CSTS_results))))

AC_results:
0.31302850098791185
TS_results:
0.05689427230773313
CSTS_results:
0.07597527208201493


In [155]:
print(acc_results)

[0.651, 0.5639, 0.4721, 0.438, 0.3986, 0.6829, 0.6401, 0.5331, 0.4997, 0.425, 0.6382, 0.581, 0.5246, 0.4258, 0.3512, 0.6961, 0.6573, 0.5855, 0.503, 0.3783, 0.3993, 0.4042, 0.4291, 0.3289, 0.3513, 0.5843, 0.5045, 0.4507, 0.4459, 0.401, 0.5696, 0.5354, 0.4922, 0.4521, 0.3986, 0.6483, 0.5721, 0.5854, 0.5653, 0.54, 0.6728, 0.6226, 0.5513, 0.5409, 0.4748, 0.6854, 0.6458, 0.6054, 0.5586, 0.4614, 0.7047, 0.6975, 0.6886, 0.6744, 0.6319, 0.6767, 0.5734, 0.494, 0.3783, 0.1989, 0.6043, 0.5982, 0.5544, 0.4916, 0.4712, 0.6889, 0.6688, 0.6508, 0.5686, 0.4554, 0.6796, 0.637, 0.6214, 0.5974, 0.5686, 0.6801, 0.6137, 0.5785, 0.5091, 0.447, 0.6964, 0.585, 0.5002, 0.4219, 0.3055, 0.6712, 0.6143, 0.5644, 0.6253, 0.5731, 0.6752, 0.6559, 0.6974, 0.6676, 0.623]


In [156]:
print(AC_results)

[0.8768938289047543, 0.837108389877433, 0.8042482551633682, 0.7924244916383646, 0.7841384748680134, 0.8923598882268036, 0.8734436809775249, 0.8246622929684018, 0.8121906407070694, 0.7910475813026588, 0.8913638637234937, 0.8652425629593042, 0.8376033932662527, 0.7951425246138926, 0.7699623924260166, 0.9086157761566732, 0.9007210888659134, 0.8880728840496118, 0.8758626623043698, 0.8558056324709005, 0.8212118099296088, 0.8268701607727522, 0.825253173017265, 0.8181323275274984, 0.8087014787414917, 0.8949403192655, 0.8828286250172336, 0.8741018981876794, 0.8717841209665826, 0.8672806384667616, 0.8791042151836198, 0.877514365111139, 0.8705219892996149, 0.8658454850547948, 0.8545418469717173, 0.8936421798720414, 0.8581827377220492, 0.8671515599874541, 0.862831326866085, 0.8446643210122553, 0.898355460297235, 0.8780508560404494, 0.8517011558034937, 0.8488903200420435, 0.8157444346081698, 0.9088944492569421, 0.8982663396440106, 0.8891245210000807, 0.8791414812142608, 0.8502583650248956, 0.91166

In [157]:
print(TS_results)

[0.6272623038604255, 0.5534199576127247, 0.49592856660379786, 0.4755811393509783, 0.46090196067867084, 0.6624951853480968, 0.6209175830317167, 0.5360764646645102, 0.5105972855138613, 0.47481541594331056, 0.6566998855214617, 0.5978708364307403, 0.5449883124868313, 0.4683213877329503, 0.4287328663616697, 0.7025474934983739, 0.6833189664335477, 0.6521146832761106, 0.6150350974622916, 0.5584527172376893, 0.5180443906866569, 0.5242117509024344, 0.5274041495082346, 0.5030763251929926, 0.4983216465171991, 0.6642034484289119, 0.6311107500413472, 0.604594500465856, 0.6049609087068497, 0.582438374948789, 0.628881696574335, 0.6234775761340446, 0.6025660755026805, 0.5911144536464235, 0.5664231266614487, 0.662864360884118, 0.5851538101902953, 0.6050592338526403, 0.5944865863279252, 0.559570571040717, 0.6751283623859788, 0.631670222704502, 0.5792951566608936, 0.5720105286845472, 0.5215285802284169, 0.7022056279048705, 0.6789984217373237, 0.6541895241185818, 0.6264893006024916, 0.5645988002540947, 0.

In [158]:
print(CSTS_results)

[0.5515593057758293, 0.5147871343833836, 0.5008149062271926, 0.5016209112176129, 0.5098223481403296, 0.5763804677074243, 0.5518015893418705, 0.5098372453085749, 0.5025061229433184, 0.5047246561309477, 0.5223176941558256, 0.47345182921535317, 0.4470981478264821, 0.4315057919396966, 0.4466172395773745, 0.5811998603580886, 0.5556437526609505, 0.5169991870143054, 0.4818816601523879, 0.4352473023008024, 0.45943552009165123, 0.44753141161053656, 0.44766669495310873, 0.44274276999000867, 0.44019929037609135, 0.5091688882283092, 0.459974554584511, 0.4249838528141137, 0.42493304953866506, 0.3970805145601816, 0.489504463968607, 0.47996731501157175, 0.4572845857920349, 0.4423207490840934, 0.41620414822817564, 0.5348822535930253, 0.5050190742134401, 0.498455324599794, 0.48891912479360455, 0.4786520633072486, 0.5587701566827356, 0.517923318977829, 0.4814772486880941, 0.4819828871422228, 0.4611307330685745, 0.5754944558670185, 0.5423573671898654, 0.5126516575246939, 0.4861869498490887, 0.44539377569

In [185]:
learnTs = [4.71686642,  2.98959056,  2.77322311,  5.46932655,  3.80400751,
        1.88290618,  3.47795797,  2.94641951,  1.57841305,  2.1682578]
learnBs = [-0.20477415,  0.00760388,  0.17515667, -0.11472634, -0.05703332,
        0.24030254, -0.12796915, -0.05191982,  0.11665848,  0.016691]

In [186]:
# bring a test condition here.
acc_results = []
AC_results = []
TS_results = []
CSTS_results = []
cifar_resultsdir = '/vol/biomedic3/zl9518/ModelEvaluation/LDAM-DRW/cifar10results/'
corruptions = ['gaussian_noise', 'shot_noise' , 'impulse_noise', 'defocus_blur', 'glass_blur', 
               'motion_blur', 'zoom_blur', 'snow', 'frost', 'fog', 'brightness', 'contrast' , 'elastic_transform', 
               'pixelate', 'jpeg_compression', 'speckle_noise', 'gaussian_blur', 'spatter', 'saturate']

for cname in tqdm(corruptions):
    csvfilename = cifar_resultsdir + 'predictions_val_' + cname + '.csv'
    cnn_pred_all = pd.read_csv(csvfilename)
    for severity in range(5):
        cnn_pred = cnn_pred_all.iloc[severity * 10000:(severity + 1) * 10000, :]
        
        kcls = 10
        targets_all = np.array(cnn_pred[['target_0', 'target_1', 'target_2', 'target_3', 'target_4', 'target_5', 'target_6', 'target_7', 'target_8', 'target_9']])
        logit_all = np.array(cnn_pred[['logit_0', 'logit_1', 'logit_2', 'logit_3', 'logit_4', 'logit_5', 'logit_6', 'logit_7', 'logit_8', 'logit_9']])
        preds_all_argmax = np.argmax(logit_all, axis = 1)
        # acc
        target_class = np.argmax(targets_all, axis = 1)
        pred_class = np.argmax(logit_all, axis = 1)
        acc = np.sum(pred_class == target_class) / len(target_class)
        prob = softmax(logit_all.transpose(), T = 1).transpose()
        probmax = np.max(prob, axis = 1)
        prob_Topt = softmax(logit_all.transpose(), T = learnT).transpose()
        prob_Toptmax = np.max(prob_Topt, axis = 1)
        acc_results.append(acc)
        AC_results.append(np.mean(probmax))
        TS_results.append(np.mean(prob_Toptmax))
        targets_all = []
        preds_class_all = []
        for label in range(kcls):
            preds_all = softmax(logit_all.transpose(), T = learnTs[label], b = learnBs[label]).transpose()
            preds_all_max = np.max(preds_all, axis = 1)
            targets_y1 = np.where(preds_all_argmax==label)[0]
            preds_class = preds_all_max[targets_y1]

            preds_class_all = np.concatenate((preds_class_all, preds_class), axis=0)
        CSTS_results.append(np.mean(preds_class_all))

100%|████████████████████████████| 19/19 [00:06<00:00,  2.74it/s]


In [187]:
print('AC_results:')
print(np.mean(np.abs(np.array(acc_results)-np.array(AC_results))))
print('TS_results:')
print(np.mean(np.abs(np.array(acc_results)-np.array(TS_results))))
print('CSTS_results:')
print(np.mean(np.abs(np.array(acc_results)-np.array(CSTS_results))))

AC_results:
0.31302850098791185
TS_results:
0.05689427230773313
CSTS_results:
0.03841334508990734


In [188]:
print(CSTS_results)

[0.5875788947728858, 0.5204646531675186, 0.4654268375139067, 0.44335277651920135, 0.4284824833734593, 0.6212016752819854, 0.5842045232670057, 0.5072186050929492, 0.4809168356988884, 0.44418804101216447, 0.5960022621167216, 0.5352650943126578, 0.48645329170128254, 0.41564778213845155, 0.3821654984401672, 0.6462742031422023, 0.617677558593075, 0.5714736959419754, 0.5162749238165109, 0.42896973750342715, 0.45234401104659866, 0.45325999394406724, 0.46037321316891855, 0.426001794653303, 0.42908892647841423, 0.5871628564833832, 0.5380724314977314, 0.4997654420433724, 0.4996178540931898, 0.4663074006256311, 0.556290736602417, 0.5359314914204976, 0.5054922742511131, 0.4810467459667444, 0.446541279507937, 0.6003385525162502, 0.5275727557680069, 0.5397518581150964, 0.5245830364042148, 0.49956673276257313, 0.6201225700450186, 0.575795161417209, 0.5284962974419752, 0.5224225208615519, 0.48133309046367095, 0.6435570062770808, 0.6113541580493025, 0.5771491573956364, 0.539903937504163, 0.463890144147